<h2 id="k-means">K-means</h2>

<p><a href="http://en.wikipedia.org/wiki/K-means_clustering">k-means</a> is one of the
most commonly used clustering algorithms that clusters the data points into a
predefined number of clusters. The MLlib implementation includes a parallelized
variant of the <a href="http://en.wikipedia.org/wiki/K-means%2B%2B">k-means++</a> method
called <a href="http://theory.stanford.edu/~sergei/papers/vldb12-kmpar.pdf">kmeans||</a>.</p>

<p><code>KMeans</code> is implemented as an <code>Estimator</code> and generates a <code>KMeansModel</code> as the base model.</p>

<h3 id="input-columns">Input Columns</h3>

<table class="table">
  <thead>
    <tr>
      <th align="left">Param name</th>
      <th align="left">Type(s)</th>
      <th align="left">Default</th>
      <th align="left">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>featuresCol</td>
      <td>Vector</td>
      <td>"features"</td>
      <td>Feature vector</td>
    </tr>
  </tbody>
</table>

<h3 id="output-columns">Output Columns</h3>

<table class="table">
  <thead>
    <tr>
      <th align="left">Param name</th>
      <th align="left">Type(s)</th>
      <th align="left">Default</th>
      <th align="left">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>predictionCol</td>
      <td>Int</td>
      <td>"prediction"</td>
      <td>Predicted cluster center</td>
    </tr>
  </tbody>
</table>

In [1]:
import findspark
findspark.init('Path_to_Spark_Installation')

### Clustering Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

data = spark.read.csv('hack_data.csv',
                      header = True,
                     inferSchema = True)
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [3]:
data.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)]

#### Format the Data

In [4]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [5]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used',
                                         'Servers_Corrupted', 'Pages_Corrupted', 'WPM_Typing_Speed'],
                           outputCol = 'features')
final_data = assembler.transform(data)

#### Scale the Data

In [6]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol = 'features', 
                        outputCol = 'scaledFeatures', 
                        withStd = True, 
                        withMean = True)
scaled_data = scaler.fit(final_data).transform(final_data)

#### Train the Model and Evaluate

In [7]:
# Let's check with K = 2 and K =3. Since that is what we suspect

from pyspark.ml.clustering import KMeans

kmeans_2 = KMeans(k = 2, featuresCol = 'scaledFeatures')
kmeans_3 = KMeans(k = 3, featuresCol = 'scaledFeatures')

# Fitting the KMeans model with the data
model_2 = kmeans_2.fit(scaled_data)
model_3 = kmeans_3.fit(scaled_data)

In [8]:
# Evaluating the performance of the model(Within Set Sum of Squared Error)
print('WSSE with K = 2')
print(model_2.computeCost(scaled_data))
print('--'*15)
print('WSSE with K = 3')
print(model_3.computeCost(scaled_data))

WSSE with K = 2
601.7707512676687
------------------------------
WSSE with K = 3
434.7550730848762


*** This does not help to decide how many hackers were there because it is obvious to have the WSEE decreased with increased number  of cluster***

*** Based on the fact that each hackers have roughly the same amount of attacks, lets check how many hacks belongs to each hacker. We will try to find for which value of K do we get the same or similar count ***

In [9]:
# to check which point belongs to which cluster(because we can not visualize higher-dimension data)
results_2 = model_2.transform(scaled_data) 
print('Clustering with K = 2')
results_2.drop('features', 'scaledFeatures', 'Location').show()

results_3 = model_3.transform(scaled_data) 
print('Clustering with K = 3')
results_3.drop('features', 'scaledFeatures', 'Location').show()

Clustering with K = 2
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+----------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|WPM_Typing_Speed|prediction|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+----------+
|                    8.0|           391.09|              1|             2.96|            7.0|           72.37|         0|
|                   20.0|           720.99|              0|             3.04|            9.0|           69.08|         0|
|                   31.0|           356.32|              1|             3.71|            8.0|           70.58|         0|
|                    2.0|           228.08|              1|             2.48|            8.0|            70.8|         0|
|                   20.0|            408.5|              0|             3.57|            8.0|           71.28|         0|
| 

In [10]:
# Let's count the number of record that belongs to each hacker

print('Number of hacks by each hacker when K = 2')
results_2.groupBy('prediction').count().show()

print('\nNumber of hacks by each hacker when K = 3')
results_3.groupBy('prediction').count().show()

Number of hacks by each hacker when K = 2
+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+


Number of hacks by each hacker when K = 3
+----------+-----+
|prediction|count|
+----------+-----+
|         1|   88|
|         2|   79|
|         0|  167|
+----------+-----+



*** Based on this analysis and above mentined fact, we can conclude that there are actually just 2 hackers ***